### Import Libraries

In [22]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader , Dataset

from sklearn.preprocessing import StandardScaler

### Load The Dataset

In [3]:
df = pd.read_csv("/content/drive/MyDrive/diabetes.csv")
df.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Number of times pregnant      768 non-null    int64  
 1   Plasma glucose concentration  768 non-null    int64  
 2   Diastolic blood pressure      768 non-null    int64  
 3   Triceps skin fold thickness   768 non-null    int64  
 4   2-Hour serum insulin          768 non-null    int64  
 5   Body mass index               768 non-null    float64
 6   Age                           768 non-null    int64  
 7   Class                         768 non-null    object 
dtypes: float64(1), int64(6), object(1)
memory usage: 48.1+ KB


In [5]:
df.isnull().sum()

Number of times pregnant        0
Plasma glucose concentration    0
Diastolic blood pressure        0
Triceps skin fold thickness     0
2-Hour serum insulin            0
Body mass index                 0
Age                             0
Class                           0
dtype: int64

In [6]:
df.describe()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,33.240885
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,11.760232
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,24.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,29.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,41.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,81.000000


In [10]:
x = df.iloc[:,:6]
y = df["Class"]

In [11]:
x.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index
0,6,148,72,35,0,33.6
1,1,85,66,29,0,26.6
2,8,183,64,0,0,23.3
3,1,89,66,23,94,28.1
4,0,137,40,35,168,43.1


In [12]:
y.head()

0    positive
1    negative
2    positive
3    negative
4    positive
Name: Class, dtype: object

In [13]:
y = y.map({"positive":1,"negative":0})

In [14]:
y.head(10)

0    1
1    0
2    1
3    0
4    1
5    0
6    1
7    0
8    1
9    1
Name: Class, dtype: int64

In [15]:
y = np.array(y,dtype="float64")

In [17]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [20]:
x = torch.tensor(x)
y = torch.tensor(y).unsqueeze(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [21]:
x.shape , y.shape

(torch.Size([768, 6]), torch.Size([768, 1]))

### Creating Custom Dataset

In [29]:
class Dataset(Dataset):

  def __init__(self,x,y):
    self.x = x
    self.y = y

  def __getitem__(self,index):
    return self.x[index],self.y[index]

  def __len__(self):
    return len(self.x)


In [30]:
dataset = Dataset(x,y)

In [31]:
len(dataset)

32

In [32]:
train_loader = DataLoader(dataset=dataset,batch_size=32,shuffle=True)

In [34]:
for x,y in train_loader:
  print(x.shape,y.shape)
  break

torch.Size([32, 6]) torch.Size([32, 1])


### Creating ANN Model

In [53]:
class ANN(nn.Module):
  def __init__(self,input_features,output_features):
    super(ANN,self).__init__()

    self.fc1 = nn.Linear(input_features ,5)
    self.fc2 = nn.Linear(5,4)
    self.fc3 = nn.Linear(4,3)
    self.fc4 = nn.Linear(3,output_features)
    self.sigmoid = nn.Sigmoid()
    self.relu = nn.ReLU()

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.relu(out)
    out = self.fc3(out)
    out = self.relu(out)
    out = self.fc4(out)
    out = self.sigmoid(out)

    return out

In [54]:
net = ANN(6,1)

loss_fn = torch.nn.BCELoss()
optimizer = optim.SGD(net.parameters(),lr=0.001)

### Trainig ANN Model

In [58]:
epochs = 100

for epoch in range(epochs):

  train_loss = 0.0
  train_acc = 0.0

  for inputs,labels in train_loader:
    
    inputs = inputs.float()
    labels = labels.float()


    outputs = net(inputs) # forward
    loss = loss_fn(outputs,labels)

    train_loss += loss.item()

    optimizer.zero_grad() 
    loss.backward() # Wn = Wo - lr * gradient
    optimizer.step()


    output = (outputs>0.5).float()
    train_acc+= (output==labels).sum().item()

  print("Epoch{} , Training Acc {:.2f} , Training Loss{:.2f}".format(epoch+1, 
                              train_acc/len(x),train_loss/len(train_loader)))

Epoch1 , Training Acc 0.62 , Training Loss0.68
Epoch2 , Training Acc 0.62 , Training Loss0.68
Epoch3 , Training Acc 0.62 , Training Loss0.68
Epoch4 , Training Acc 0.62 , Training Loss0.68
Epoch5 , Training Acc 0.62 , Training Loss0.68
Epoch6 , Training Acc 0.62 , Training Loss0.68
Epoch7 , Training Acc 0.62 , Training Loss0.68
Epoch8 , Training Acc 0.62 , Training Loss0.68
Epoch9 , Training Acc 0.62 , Training Loss0.68
Epoch10 , Training Acc 0.62 , Training Loss0.68
Epoch11 , Training Acc 0.62 , Training Loss0.68
Epoch12 , Training Acc 0.62 , Training Loss0.68
Epoch13 , Training Acc 0.62 , Training Loss0.68
Epoch14 , Training Acc 0.62 , Training Loss0.68
Epoch15 , Training Acc 0.62 , Training Loss0.68
Epoch16 , Training Acc 0.62 , Training Loss0.68
Epoch17 , Training Acc 0.62 , Training Loss0.68
Epoch18 , Training Acc 0.62 , Training Loss0.68
Epoch19 , Training Acc 0.62 , Training Loss0.68
Epoch20 , Training Acc 0.62 , Training Loss0.68
Epoch21 , Training Acc 0.62 , Training Loss0.68
E